In [1]:
CPU_CORES = 51
addprocs(CPU_CORES - 1);
@everywhere using DistributedArrays, JuMP, Distributions, Ipopt
using StatsBase

In [2]:
print(nprocs(),"\n")
print(nworkers(),"\n")
print(workers(),"\n")

51
50
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [3]:
@everywhere landscape1= [0. 0 0 2 2 0 0 0 2 0 0 2;
              2 0 1 2 5 5 0 0 2 0 0 0;
              3 1 2 2 1 1 2 0 0 3 0 0;
              2 1 1 2 1 2 2 0 3 4 4 0
              2 2 1 1 1 1 1 0 3 4 4 3;
              2 2 1 1 1 1 1 1 3 3 3 0;
              0 2 0 1 5 5 5 0 1 1 0 3;
              3 0 0 1 0 0 2 0 0 3 1 3;
              3 4 0 1 1 3 2 2 2 3 3 3;
              4 3 2 0 0 3 2 2 2 3 2 2] + 1
@everywhere Ncell = 30

In [4]:
@everywhere function create_pixel_mat(landscape, ordered_vec)
    retmat = deepcopy(landscape)
    for i = 1:length(ordered_vec)
        retmat[abs.(retmat - i) .< 0.000000001] = ordered_vec[i]
    end
    
    if size(retmat) == (10,12)
        retmat_zone = hcat(reshape(retmat[1:5 , 1:6],Ncell),
            reshape(retmat[1:5 , 7:12],Ncell),
            reshape(retmat[6:10 , 1:6],Ncell),
            reshape(retmat[6:10 , 7:12],Ncell))
    end
    retmat_zone
end


In [5]:
# Using the prior and visibility info given in the DCA paper
# These have @everywhere because they are fixed. Better declare them in each process rather than
# copying from the parent process
@everywhere s1 = create_pixel_mat(landscape1, [0.4,0.9,0.3,0.2,0.1,0.8])
@everywhere s2 = create_pixel_mat(landscape1, [0.5,0.1,0.1,0.7,0.6,0.9])
@everywhere s3 = create_pixel_mat(landscape1, [0.6,0.1,0.4,0.8,0.7,0.1])
@everywhere s4 = create_pixel_mat(landscape1, [0.8,0.1,0.6,0.2,0.1,0.7])
@everywhere s5 = create_pixel_mat(landscape1, [0.5,0.3,0.5,0.4,0.3,0.6])
@everywhere s6 = create_pixel_mat(landscape1, [0.1,0.5,0.2,0.6,0.5,0.2])




@everywhere Z = 4 # number of zones
@everywhere S = 6 # number of sensors
W = reshape(hcat(s1,s2,s3,s4,s5,s6),Ncell,Z,S) # visibility tensor

# Decrease censors' visibility
W = W/20



@everywhere θ = 0.3
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
@everywhere epsilon = 1e-5


# No @everywhere because these are only used by the parent process
maxiter = 15 # number of iterations for 



# No @everywhere because these are changing, pass into functions as parameters

Φ = 5
# prior_ordered_vec = [0.0085,0.001,0.0115,0.013,0.014,0] # prior of target, order given in the paper
# prior = create_pixel_mat(landscape1, prior_ordered_vec)
# print(sum(prior))





5

In [6]:



####################################
## CE Algorithm parallel N

@everywhere function solve_alloc(u, Φ,prior,W)

    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S]) # resource allocation for each senor in each zone in each cell
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    getobjectivevalue(m_ϕ), transpose(u)
end
    
    

function CE_parallel(Φ,prior,W)
    
    P = ones(S,Z)/Z
    ind = true
    P_holder = zeros(S,Z)
    for iter=1:maxiter

        cumsumP = cumsum(P,2)
        
        vec_tile = rand(Uniform(0,1), S * N,1)
        cumsumP_tile = repmat(cumsumP,N)
        which_zone = floor.(findmax(cumsumP_tile .> vec_tile,2)[2] / (S*N) - epsilon) + 1
        u = abs.(repmat(transpose(1:Z),S*N,1) .- which_zone) .< epsilon
        u = transpose(u)
        
        arr_obj = []
        arr_assign = zeros(S,Z,N)
#         tic()
        retobj = @DArray [solve_alloc(u[:,(i-1)*S+1:i*S],Φ,prior,W) for i = 1:N]
#         toc()
        for i = 1:N
            push!(arr_obj, retobj[i][1])
            arr_assign[:,:,i] = retobj[i][2]
        end
 
        best_ind = sortperm(arr_obj)[1:Threshold]
        P = squeeze(sum(arr_assign[:,:,best_ind],3)/Threshold,3)
        
        if sum(abs.(P - 1) .<=0.000001) == S
#             print(iter,'\n')
            break
        end
    end
    
    u = transpose(P)
    m_ϕ = Model(solver = IpoptSolver(print_level=0))

    @variable(m_ϕ,ϕvar[1:Ncell,1:Z,1:S])
    @variable(m_ϕ,exponential[1:Ncell,1:Z])
    @variable(m_ϕ, aux[1:Ncell,1:Z])
    @objective(m_ϕ, Min, sum(prior .* exponential))
    @NLconstraint(m_ϕ, [i=1:Ncell,j=1:Z], exp(aux[i,j]) <= exponential[i,j])
    @constraint(m_ϕ,aux .== squeeze(-sum(W .* ϕvar .* reshape(u,1,Z,S),3),3))
    @constraint(m_ϕ, exponential .>= 0)
    @constraint(m_ϕ,ϕvar .>= 0)
    @constraint(m_ϕ,[i=1:Z,j=1:S],sum(ϕvar[:,i,j]) <= Φ)

    solve(m_ϕ)
    
    getobjectivevalue(m_ϕ), P, exp.(getvalue(aux)), getvalue(ϕvar)
    
    
    
end


CE_parallel (generic function with 1 method)

In [7]:
function map2repr(x,y)
    xoffset = [0,6,0,6]
    yoffset = [0,0,5,5]
    which_zone = 1 + (x > centerx)  + (y > centery) * 2
    xoff = x - xoffset[which_zone]
    yoff = y - yoffset[which_zone]
    which_cell = (xoff-1) * 5 + yoff
    return which_cell, which_zone
end


# Make prior at the centre of the landscape
# raidus 1, 50% 
# radius 2, 50%



prior = zeros(30,4)
radius = 2
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 12
    end
end


radius = 1
centerx = 6.5
centery = 5.5
maxx = convert(Int64,centerx + (radius - 0.5))
minx = convert(Int64,centerx - (radius - 0.5))
maxy = convert(Int64,centery + (radius - 0.5))
miny = convert(Int64,centery - (radius - 0.5))

for x=minx:maxx
    for y=miny:maxy
        which_cell, which_zone = map2repr(x,y)
        prior[which_cell,which_zone] = 0.5 / 4
    end
end

In [8]:
# Test run. good practice to just run it when first open the notebook (initialize the optimizer for each process)

tic()
Φ = 5
maxiter = 20
@everywhere N = 30 # Threshold at every worker needs it
@everywhere Threshold = convert(Int64,floor(N * θ)) # for accepting P computed from each sample
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,prior,W)
toc()
print(obj,'\n')

print(assignment,'\n')

	From worker 22:	
	From worker 22:	******************************************************************************
	From worker 22:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 22:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 22:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 22:	******************************************************************************
	From worker 22:	
	From worker 9:	
	From worker 9:	******************************************************************************
	From worker 9:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 9:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 9:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 9:	******************************************************************************
	From work

	From worker 11:	
	From worker 11:	******************************************************************************
	From worker 11:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 11:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 11:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 11:	******************************************************************************
	From worker 11:	
	From worker 24:	
	From worker 24:	******************************************************************************
	From worker 24:	This program contains Ipopt, a library for large-scale nonlinear optimization.
	From worker 24:	 Ipopt is released as open source code under the Eclipse Public License (EPL).
	From worker 24:	         For more information visit http://projects.coin-or.org/Ipopt
	From worker 24:	******************************************************************************
	Fro

In [ ]:
##############################################
## Moving Target

In [9]:
# Assume Ncell,Z exist, current ONLY WORK WITH THE LANDSCAPE IN PAPERS
# construct a 30x4x30x4 one step moving probability tensor

function build_c2c_tensor(matrix)
    

    
    # first, construct 120x120 one step transition matrix

    # here we number all cells from 1 to 120 in a row-major fashion
    c2c = zeros(Ncell * Z, Ncell * Z)
    for i = 1:(Ncell * Z)
        # Stay in current cell
        c2c[i,i] = matrix[2,2]

        # move right
        if mod(i,12) ==  0 # right end 
            c2c[i,i] = c2c[i,i] + matrix[2,3]
        else
            c2c[i,i+1] = matrix[2,3]
        end


        # move left
        if mod(i,12) ==  1 # left end 
            c2c[i,i] = c2c[i,i] + matrix[2,1]
        else
            c2c[i,i-1] = matrix[2,1]
        end

        # move down
        if i > 12 * (10-1) # bottom end
            c2c[i,i] = c2c[i,i] + matrix[3,2]
        else
            c2c[i,i+12] = matrix[3,2]
        end

        # move up
        if i <= 12 # top row
            c2c[i,i] = c2c[i,i] + matrix[1,2]
        else
            c2c[i,i-12] = matrix[1,2]
        end



        # move diagonal bottom-right
        if (mod(i,12) == 0) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,3]
        else
            c2c[i,i+12+1] = matrix[3,3]
        end


        # move diagonal top-right
        if (mod(i,12) == 0) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,3]
        else
            c2c[i,i-12+1] = matrix[1,3]
        end

        # move diagonal top-left
        if (mod(i,12) == 1) | (i <= 12)
            c2c[i,i] = c2c[i,i] + matrix[1,1]
        else
            c2c[i,i-12-1] = matrix[1,1]
        end

        # move diagonal bottom-left
        if (mod(i,12) == 1) | (i > 12 * (10-1))
            c2c[i,i] = c2c[i,i] + matrix[3,1]
        else
            c2c[i,i+12-1] = matrix[3,1]
        end


    end

    # convert the 120x120 matrix into 30x4x30x4 tensor
    # Note that Julia is column major: suppose the first (leftmost, upmost) cell
    # is cell 1, cell 2 refers to the cell below cell 1 not the cell to the right
    # This affects how cell index is assigned
    c2c_tensor = zeros(Ncell,Z,Ncell,Z)
    for i=1:Ncell*Z
        zone_prev = (i / 60 > 1) * 2 + ((mod(i,12) / 6 > 1) | (mod(i,12) == 0)) + 1
        offset_prev = [0,6,60,66][zone_prev]
        cell_prev = (rem(i-offset_prev,12) - 1) * 5 + div(i-offset_prev,12) + 1 # column major in julia
        for j=1:Ncell*Z
            zone_cur = (j / 60 > 1) * 2 + ((mod(j,12) / 6 > 1) | (mod(j,12) == 0)) + 1
            offset_cur = [0,6,60,66][zone_cur]
            cell_cur = (rem(j-offset_cur,12) - 1) * 5 + div(j-offset_cur,12) + 1 # column major in julia
            c2c_tensor[cell_prev,zone_prev,cell_cur,zone_cur] = c2c[i,j]
        end
    end
    
    return c2c_tensor
end

build_c2c_tensor (generic function with 1 method)

In [ ]:
# # Assume Ncell,Z exist, current ONLY WORK WITH THE LANDSCAPE IN PAPERS
# # construct a 30x4x30x4 one step moving probability tensor

# function build_c2c_tensor_JUMP(matrix)
    

    
#     # first, construct 120x120 one step transition matrix

#     # here we number all cells from 1 to 120 in a row-major fashion
#     c2c = zeros(Ncell * Z, Ncell * Z)
#     for i = 1:(Ncell * Z)
#         # Stay in current cell
#         c2c[i,i] = matrix[2,2]

#         # move right
#         if mod(i,12) ==  0 # right end 
#             c2c[i,i] = c2c[i,i] + matrix[2,3]
#         else
#             c2c[i,i+1] = matrix[2,3]
#         end


#         # move left
#         if mod(i,12) ==  1 # left end 
#             c2c[i,i] = c2c[i,i] + matrix[2,1]
#         else
#             c2c[i,i-1] = matrix[2,1]
#         end

#         # move down
#         if i > 12 * (10-1) # bottom end
#             c2c[i,i] = c2c[i,i] + matrix[3,2]
#         else
#             c2c[i,i+12] = matrix[3,2]
#         end

#         # move up
#         if i <= 12 # top row
#             c2c[i,i] = c2c[i,i] + matrix[1,2]
#         else
#             c2c[i,i-12] = matrix[1,2]
#         end



#         # move diagonal bottom-right
#         if (mod(i,12) == 0) | (i > 12 * (10-1))
#             c2c[i,i] = c2c[i,i] + matrix[3,3]
#         else
#             c2c[i,i+12+1] = matrix[3,3]
#         end


#         # move diagonal top-right
#         if (mod(i,12) == 0) | (i <= 12)
#             c2c[i,i] = c2c[i,i] + matrix[1,3]
#         else
#             c2c[i,i-12+1] = matrix[1,3]
#         end

#         # move diagonal top-left
#         if (mod(i,12) == 1) | (i <= 12)
#             c2c[i,i] = c2c[i,i] + matrix[1,1]
#         else
#             c2c[i,i-12-1] = matrix[1,1]
#         end

#         # move diagonal bottom-left
#         if (mod(i,12) == 1) | (i > 12 * (10-1))
#             c2c[i,i] = c2c[i,i] + matrix[3,1]
#         else
#             c2c[i,i+12-1] = matrix[3,1]
#         end


#     end

#     # convert the 120x120 matrix into 30x4x30x4 tensor
#     # Note that Julia is column major: suppose the first (leftmost, upmost) cell
#     # is cell 1, cell 2 refers to the cell below cell 1 not the cell to the right
#     # This affects how cell index is assigned
#     c2c_tensor = zeros(Ncell,Z,Ncell,Z)
#     for i=1:Ncell*Z
#         zone_prev = (i / 60 > 1) * 2 + ((mod(i,12) / 6 > 1) | (mod(i,12) == 0)) + 1
#         offset_prev = [0,6,60,66][zone_prev]
#         cell_prev = (rem(i-offset_prev,12) - 1) * 5 + div(i-offset_prev,12) + 1 # column major in julia
#         for j=1:Ncell*Z
#             zone_cur = (j / 60 > 1) * 2 + ((mod(j,12) / 6 > 1) | (mod(j,12) == 0)) + 1
#             offset_cur = [0,6,60,66][zone_cur]
#             cell_cur = (rem(j-offset_cur,12) - 1) * 5 + div(j-offset_cur,12) + 1 # column major in julia
#             c2c_tensor[cell_prev,zone_prev,cell_cur,zone_cur] = c2c[i,j]
#         end
#     end
    
#     return c2c_tensor
# end

In [ ]:
# Myopic and Optimal Search Plan (without cross-cueing)

In [ ]:
T = 4
Φ = 5
Total_Iter = 1 # the number of iterations to go from myopic search plan to optimal search plan

In [10]:


function mov_target_CE(prior,W,c2c_tensor)
    DTensor = ones(Ncell,Z,T)
    moving_target_obj_vec = zeros(Total_Iter,T)
    prob_nondetect_Tensor = zeros(Ncell,Z,T)
    final_assignment = zeros(S,Z,T)
    final_res_alloc = zeros(Ncell,Z,S,T)
    
    
    for i = 1:Total_Iter
        # Initialize UTensor
        UTensor = zeros(Ncell,Z,T)
        UTensor[:,:,1] =  prior


        for τ = 1:T
            if τ > 1
                UTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
            end
            β = UTensor[:,:,τ] .* DTensor[:,:,τ]
            obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,β,W)
            final_assignment[:,:,τ] = assignment
            final_res_alloc[:,:,:,τ] = res_alloc
            prob_nondetect_Tensor[:,:,τ] = prob_nondetect
            moving_target_obj_vec[i,τ] = obj
            print("Iter: ",i,". Completed timestep: ",τ,". Obj: ",obj,"\n")
        end

        # Compute the DTensors using the nondetection probabilities at each timestep as calculated
        # in the previous iteration
        for τ = T-1:-1:1
            DTensor[:,:,τ] = sum(sum(c2c_tensor .* 
                    reshape(prob_nondetect_Tensor[:,:,τ+1],1,1,Ncell,Z) .* 
                    reshape(DTensor[:,:,τ+1],1,1,Ncell,Z),3),4)
        end
    end
    
    return moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc

end



    
function compute_non_detection_test(prior, W, c2c_tensor,prob_nondetection_tensor,T)
    # Compute nondetection probability at each time step (used in computing U and D)



    # Compute U and D

    UTensor_holder = zeros(Ncell,Z,T)
    UTensor_holder[:,:,1] =  prior
    for i=2:T

        UTensor_holder[:,:,i] = sum(sum(c2c_tensor .* 
            reshape(prob_nondetection_tensor[:,:,i-1],Ncell,Z,1,1) .* 
            reshape(UTensor_holder[:,:,i-1],Ncell,Z,1,1),1),2)

    end


    β_holder = UTensor_holder
    return sum(sum(β_holder[:,:,:] .* prob_nondetection_tensor[:,:,:],1),2)
end 




compute_non_detection_test (generic function with 1 method)

In [46]:

# moving_mat_no= [1/9 1/9 1/9;
#                  1/9 1/9 1/9;
#                  1/9 1/9 1/9]
# c2c_tensor_init = build_c2c_tensor(moving_mat_no)

# sum(moving_mat_no)

moving_mat= [0.25 0.125 0.0;
             0.125 0.5 0.0;
             0.0 0.0 0.0]
c2c_tensor_true = build_c2c_tensor(moving_mat)




moving_mat_oppo= [0 0 0;
                  0 0.5 0.125;
                  0 0.125 0.25]
c2c_tensor_init = build_c2c_tensor(moving_mat_oppo)
print("")

In [ ]:
# When you have the true moving probability

T = 30
tic()
moving_target_obj_vec, prob_nondetect_Tensor, final_assignment, final_res_alloc = mov_target_CE(prior,W,c2c_tensor_true)
nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
print(nondetection_no_movement_test,"\n")
toc()


In [47]:
T = 15
holder = zeros(T)
holder_expect = zeros(T)

15-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [48]:
UTensor = zeros(Ncell,Z,T)
UTensor[:,:,1] =  prior
        
UTensor_actual = zeros(Ncell,Z,T)
UTensor_actual[:,:,1] =  prior

prob_nondetect_Tensor = zeros(Ncell,Z,T)
actuals = []

0-element Array{Any,1}

In [49]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
push!(actuals,obj)
obj
holder[τ] = obj
holder_expect[τ] = obj

0.928493796181211


0.9284938007265591

In [50]:
for τ=2:15
    UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
    UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
    obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
    prob_nondetect_Tensor[:,:,τ] = prob_nondetect
    actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
    push!(actuals,actual)
    print(actual,"\n")
    print(obj,"\n")
    print(actual / obj,'\n')
    holder[τ] = actual
    holder_expect[τ] = obj

    nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
    # nondetection_no_movement_test[:,:,τ]

    num_CE_movementmet = 1000
    movement_assign = zeros(3,3,num_CE_movementmet)
    movement_obj = []
    movement_dir = zeros(num_CE_movementmet)

    for i=1:num_CE_movementmet
        gen_mat = rand(Uniform(0,1),3,3)
        if i <= 250
            moving_mat = tril(gen_mat)
            moving_mat = flipdim(moving_mat/sum(moving_mat),1)
            dir = 1
        elseif i <= 500
            moving_mat = triu(gen_mat)
            moving_mat = moving_mat /sum(moving_mat)
            dir = 2
        elseif i <= 750
            moving_mat = tril(gen_mat)
            moving_mat = moving_mat /sum(moving_mat)
            dir = 3
        else
            moving_mat = triu(gen_mat)
            moving_mat = flipdim(moving_mat/sum(moving_mat),1)
            dir = 4
        end
        c2c_tensor_trial = build_c2c_tensor(moving_mat)
        UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                                reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                                reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
    #     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
        nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
        trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
        movement_assign[:,:,i] = moving_mat
        push!(movement_obj, trial_obj)
        movement_dir[i] = dir
    end

    best_ind = sortperm(movement_obj)[1:250]
    dir_count = zeros(4)
    for i=1:4
        dir_count[i] = sum(movement_dir[best_ind] .== i)
    end

    Threshold_movement = 10

    best_dir = sortperm(dir_count)[4]
    select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

    MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

    c2c_tensor_init = build_c2c_tensor(MV)
end

0.8894470563995168
0.8563653043339374
1.0386304208007466
0.8621141515764592
0.8070808609168247
1.0681880754762514
0.836113650878019
0.7717273106233532
1.0834314651928783
0.8173104026823261
0.7436884729101674
1.0989956580664277
0.8032730510374362
0.7201080637056411
1.1154895931922109
0.7860428722809998
0.6999916627561894
1.1229317634812779
0.7747395487708453
0.6817288855723063
1.136433507757937
0.7561504510760719
0.6636847514497473
1.1393217177648627
0.7464099156397461
0.6461293412056872
1.1552020130318394
0.7371826716068272
0.6275177162505776
1.1747599350843803
0.7293962054151432
0.608794697957707
1.1980988137084834
0.7227341052837997
0.590599206864066
1.2237302334375573
0.7181619023583599
0.572273811467741
1.2549270785543234
0.7170338605695867
0.5545434490222022
1.293016555932444


In [51]:
holder

15-element Array{Float64,1}:
 0.928494
 0.889447
 0.862114
 0.836114
 0.81731 
 0.803273
 0.786043
 0.77474 
 0.75615 
 0.74641 
 0.737183
 0.729396
 0.722734
 0.718162
 0.717034

In [52]:
holder_expect

15-element Array{Float64,1}:
 0.928494
 0.856365
 0.807081
 0.771727
 0.743688
 0.720108
 0.699992
 0.681729
 0.663685
 0.646129
 0.627518
 0.608795
 0.590599
 0.572274
 0.554543

In [ ]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 10
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 11
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 12
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 13
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 14
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
τ = 15
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
push!(actuals,actual)
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

nondetection_no_movement_test = compute_non_detection_test(prior, W, c2c_tensor_true,prob_nondetect_Tensor,T)
# nondetection_no_movement_test[:,:,τ]

num_CE_movementmet = 1000
movement_assign = zeros(3,3,num_CE_movementmet)
movement_obj = []
movement_dir = zeros(num_CE_movementmet)

for i=1:num_CE_movementmet
    gen_mat = rand(Uniform(0,1),3,3)
    if i <= 250
        moving_mat = tril(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 1
    elseif i <= 500
        moving_mat = triu(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 2
    elseif i <= 750
        moving_mat = tril(gen_mat)
        moving_mat = moving_mat /sum(moving_mat)
        dir = 3
    else
        moving_mat = triu(gen_mat)
        moving_mat = flipdim(moving_mat/sum(moving_mat),1)
        dir = 4
    end
    c2c_tensor_trial = build_c2c_tensor(moving_mat)
    UTensor_trial = reshape(sum(sum(c2c_tensor_trial .* 
                            reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                            reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2),Ncell,Z)
#     trial_obj = sum(UTensor_trial .* prob_nondetect_Tensor[:,:,τ])
    nondetection_no_movement_test_trial = compute_non_detection_test(prior, W, c2c_tensor_trial,prob_nondetect_Tensor,T)
    trial_obj = sum(abs.(actuals[2:τ]  - reshape(nondetection_no_movement_test_trial[:,:,2:τ],(τ-1),1)))/(τ-1)
    movement_assign[:,:,i] = moving_mat
    push!(movement_obj, trial_obj)
    movement_dir[i] = dir
end

best_ind = sortperm(movement_obj)[1:250]
dir_count = zeros(4)
for i=1:4
    dir_count[i] = sum(movement_dir[best_ind] .== i)
end

Threshold_movement = 10

best_dir = sortperm(dir_count)[4]
select_ind = best_ind[(best_ind .>= (best_dir-1) * (num_CE_movementmet/4)) .* (best_ind .<= best_dir * (num_CE_movementmet/4))]

MV = squeeze(sum(movement_assign[:,:,select_ind[1:Threshold_movement]],3)/Threshold_movement,3)

c2c_tensor_init = build_c2c_tensor(MV)

# display(MV)
# display(movement_assign[:,:,select_ind[1:Threshold_movement]])

In [ ]:
MV

In [ ]:
holder

In [ ]:
holder_expect

In [53]:
moving_mat_no= [1/9 1/9 1/9;
                 1/9 1/9 1/9;
                 1/9 1/9 1/9]
c2c_tensor_init = build_c2c_tensor(moving_mat_no)


moving_mat_oppo= [0 0 0;
                  0 0.5 0.125;
                  0 0.125 0.25]
c2c_tensor_init = build_c2c_tensor(moving_mat_oppo)


moving_mat_oppo= [1/6 1/6 1/6;
                  1/6 1/6 0;
                  1/6 0 0]
c2c_tensor_init = build_c2c_tensor(moving_mat_oppo)


30×4×30×4 Array{Float64,4}:
[:, :, 1, 1] =
 1.0       0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 ⋮                      
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0

[:, :, 2, 1] =
 0.0       0.0  0.0  0.0
 0.666667  0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.166667  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  

In [54]:
T = 15
holder = zeros(T)
holder_expect = zeros(T)

15-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [55]:
τ = 1
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
print(sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ]),"\n")
holder[τ] = obj
holder_expect[τ] = obj
obj

0.9284937962096536


0.9284938007550035

In [56]:
τ = 2
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)
holder[τ] = actual
holder_expect[τ] = obj

0.8689934175347788
0.8748136406665273
0.9933468994295584

0.8748136406665273

In [57]:
τ = 3
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.8226114138785516
0.8342456860204006
0.986054141679356

0.8342456860204006

In [58]:
τ = 4
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.7842688371598173
0.8039387696189573
0.9755330465422598

0.8039387696189573

In [59]:
τ = 5
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.7501946434318338
0.7781298590045111
0.9640995455329066

0.7781298590045111

In [60]:
τ = 6
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.7260359444843982
0.7559307427694235
0.9604529931201067

0.7559307427694235

In [61]:
τ = 7
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.7053867299084746
0.7346736844885907
0.9601361050511795

0.7346736844885907

In [62]:
τ = 8
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.6875343449715099
0.714047047234368
0.9628698103779765

0.714047047234368

In [63]:
τ = 9
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.6611786890604691
0.6924118729787089
0.9548921889743502

0.6924118729787089

In [64]:
τ = 10
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.6339015309221032
0.6696926934709468
0.9465558413615925

0.6696926934709468

In [65]:
τ = 11
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.6001088053132263
0.6454884992791867
0.9296971301322399

0.6454884992791867

In [66]:
τ = 12
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.562430875312071
0.6189929447939373
0.9086224326826604

0.6189929447939373

In [67]:
τ = 13
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.5225217767921376
0.5905902932304887
0.884744945491703

0.5905902932304887

In [68]:
τ = 14
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)


holder[τ] = actual
holder_expect[τ] = obj

0.48243813247776796
0.5607330173312497
0.8603704750147975

0.5607330173312497

In [69]:
τ = 15
UTensor[:,:,τ] =  sum(sum(c2c_tensor_init .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor[:,:,τ-1],Ncell,Z,1,1),1),2)
UTensor_actual[:,:,τ] =  sum(sum(c2c_tensor_true .* 
                        reshape(prob_nondetect_Tensor[:,:,τ-1],Ncell,Z,1,1) .* 
                        reshape(UTensor_actual[:,:,τ-1],Ncell,Z,1,1),1),2)
obj, assignment, prob_nondetect, res_alloc = CE_parallel(Φ,UTensor[:,:,τ],W)
prob_nondetect_Tensor[:,:,τ] = prob_nondetect
actual = sum(UTensor_actual[:,:,τ] .* prob_nondetect_Tensor[:,:,τ])
print(actual,"\n")
print(obj,"\n")
print(actual / obj)

holder[τ] = actual
holder_expect[τ] = obj

0.44232079656354656
0.5298370204140452
0.8348242563682914

0.5298370204140452

In [70]:
holder

15-element Array{Float64,1}:
 0.928494
 0.868993
 0.822611
 0.784269
 0.750195
 0.726036
 0.705387
 0.687534
 0.661179
 0.633902
 0.600109
 0.562431
 0.522522
 0.482438
 0.442321

In [71]:
holder_expect

15-element Array{Float64,1}:
 0.928494
 0.874814
 0.834246
 0.803939
 0.77813 
 0.755931
 0.734674
 0.714047
 0.692412
 0.669693
 0.645488
 0.618993
 0.59059 
 0.560733
 0.529837